In [ ]:
import pandas as pd
import numpy as np
import time
import shap
import torch
import torch.nn as nn
import torch.optim  as optim
import matplotlib.pyplot as plt
import os
import random

from copy import deepcopy
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"

In [49]:
def total_dataset(weight):
    tasks = ['10', '20', '1back', '3back']

    task_datas = []
    task_labels = []

    for task in tasks:
        folder_path = './seq_data/' + weight + '/' + task +'/'
        task_data = np.load(folder_path + 'total_split_datas.npy')
        task_label = np.load(folder_path + 'total_split_labels.npy')

        ind_list = [i for i in range(len(task_data))]
        random.shuffle(ind_list)

        if task == '20':
            task_data = task_data[ind_list[:4000], :, :]
            task_label = task_label[ind_list[:4000]]
        elif task == '3back':
            task_data = task_data[ind_list[:4000], :, :]
            task_label = task_label[ind_list[:4000]]
        else:
            task_data = task_data[ind_list[:2000], :, :]
            task_label = task_label[ind_list[:2000]]
        
        task_datas.append(task_data)
        task_labels.append(task_label)
    
    total_datas = np.concatenate(task_datas, axis=0)
    total_labels = np.concatenate(task_labels, axis=0)
    
    return total_datas, total_labels

In [50]:
class MakeDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

def generation_cnn_data(device, weight = 'weight'):
    total_datas, total_labels = total_dataset(weight)
    eeg_input = torch.tensor(total_datas, dtype=float, device=device)
    eeg_output = torch.tensor(total_labels, dtype=int, device=device)
    
    return eeg_input, eeg_output

In [65]:
class CNNEEG_32(nn.Module):
    def __init__(self, input_channel, keep_batch_dim = True):
        super(CNNEEG_32, self).__init__()
        
        self.input_channel = input_channel
        self.keep_batch_dim = keep_batch_dim
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 8, 
                      stride=2, padding = 3, groups=self.input_channel),
            nn.Conv1d(self.input_channel, 128, kernel_size = 1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(128, 128, 4, stride=4, padding = 2, groups=128),
            nn.Conv1d(128, self.input_channel, kernel_size = 1)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 4, 
                      stride=4, padding = 2, groups=self.input_channel),
            nn.Conv1d(self.input_channel, self.input_channel, kernel_size = 1)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 4, 
                      stride=4, padding =0, groups=self.input_channel),
            nn.Conv1d(self.input_channel, self.input_channel, kernel_size = 1)
        )
        
        self.fc1 = nn.Linear(self.input_channel, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)
        
        self.network = nn.Sequential(self.conv1, self.conv2, self.conv3, self.conv4)
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim = 1)
    
    def Flatten(self, data):
        if self.keep_batch_dim:
            return data.view(data.size(0), -1)
        else:
            return data.view(-1)
    
    def forward(self, X):
        pred = self.network(X)
        pred = self.fc1(self.Flatten(pred))
        pred = self.fc2(pred)
        pred = self.fc3(pred)
        pred = self.softmax(pred)
        
        return pred

In [301]:
class CNNEEG_32_2(nn.Module):
    def __init__(self, input_channel, keep_batch_dim = True):
        super(CNNEEG_32_2, self).__init__()
        
        self.input_channel = input_channel
        self.keep_batch_dim = keep_batch_dim
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 10, 
                      stride=1, padding = 4, groups=self.input_channel),
            nn.Conv1d(self.input_channel, 128, kernel_size = 1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(128, 128, 8, stride=2, padding = 2, groups=128),
            nn.Conv1d(128, self.input_channel, kernel_size = 1)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 8, 
                      stride=2, padding = 3, groups=self.input_channel),
            nn.Conv1d(self.input_channel, self.input_channel, kernel_size = 1)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 4, 
                      stride=4, padding =2, groups=self.input_channel),
            nn.Conv1d(self.input_channel, self.input_channel, kernel_size = 1)
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(self.input_channel, self.input_channel, 4, 
                      stride=8, padding =0, groups=self.input_channel),
            nn.Conv1d(self.input_channel, self.input_channel, kernel_size = 1)
        )
        
        self.fc1 = nn.Linear(self.input_channel, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)
        
        self.network = nn.Sequential(self.conv1, self.conv2, self.conv3, self.conv4, self.conv5)
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim = 1)
    
    def Flatten(self, data):
        if self.keep_batch_dim:
            return data.view(data.size(0), -1)
        else:
            return data.view(-1)
    
    def forward(self, X):
        pred = self.network(X)
        pred = self.fc1(self.Flatten(pred))
        pred = self.fc2(pred)
        pred = self.fc3(pred)
        pred = self.softmax(pred)
        
        return pred

In [133]:
def restore_parameters(model, best_model):
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params

In [134]:
def train_CNNEEG(model, trainloader, epochs, lr, device):
    model.to(device)
    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    history = {'loss' : []}
    
    best_loss = np.inf
    best_model = None
    
    for epoch in range(epochs):
        losses = []
        for X, Y in trainloader:
            model.zero_grad()
            pred = model(X.float()).squeeze(1)
            loss = loss_fn(pred, Y)
            
            loss.backward()
            optimizer.step()
            losses.append(float(loss))
            
        avg_loss = np.mean(losses)
        history['loss'].append(avg_loss)
        
        print("Epoch {} / {}: Loss = {:.3f}".format(epoch+1, epochs, avg_loss))
        
        if best_loss > avg_loss:
            best_loss = avg_loss
            best_model =  deepcopy(model)
            
    restore_parameters(model, best_model)
    
    return history

In [69]:
def evaluation_eegcnn(model, train_data, test_data, batch_size, device):
    model.to(device)
    
    train_pred_list = []
    train_target_list = []
    test_pred_list = []
    test_target_list = []
    
    trainacc = 0
    testacc = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    trainlosses = []
    testlosses = []
    
    for X, y, in train_data:
        pred = model(X.float())
        loss = loss_fn(pred.squeeze(1), y)
        
        trainlosses.append(float(loss))
        train_pred_list.append(torch.argmax(pred.squeeze(1), dim = 1))
        train_target_list.append(y)
        
    trainavg_loss = np.mean(trainlosses) 
    
    for p, t in zip(train_pred_list, train_target_list):
        trainacc += torch.sum(p == t)
        
    print('Train dataset loss :', trainavg_loss)
    print('Train dataset acc :{}%'.format((trainacc / 
                                          (batch_size * len(train_data))).item() *100 ))
    
    for X, y, in test_data:
        pred = model(X.float())
        loss = loss_fn(pred.squeeze(1), y)
        
        testlosses.append(float(loss))
        test_pred_list.append(torch.argmax(pred.squeeze(1), dim = 1))
        test_target_list.append(y)
        
    testavg_loss = np.mean(testlosses)
    
    for p, t in zip(test_pred_list, test_target_list):
        testacc += torch.sum(p == t)
        
    print('Test dataset loss :', testavg_loss)
    print('Test dataset acc :{}%'.format((testacc / 
                                          (batch_size * len(test_data))).item() *100 ))
    
    return train_pred_list, train_target_list, test_pred_list, test_target_list

In [ ]:
eeg_input, eeg_output = generation_cnn_data(device)
dataset = MakeDataset(eeg_input, eeg_output)
train_dataset, test_dataset = random_split(dataset, 
                                           [round(eeg_output.size()[0]*0.8),
                                            round(eeg_output.size()[0]*0.2)])

trainloader = DataLoader(train_dataset, 
                         batch_size=256, 
                         shuffle=True)

testloader = DataLoader(test_dataset,
                        batch_size = 256,
                        shuffle=True)

In [68]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 200, 0.0001, device= device)

Epoch 1 / 200: Loss = 1.099
Epoch 2 / 200: Loss = 1.099
Epoch 3 / 200: Loss = 1.099
Epoch 4 / 200: Loss = 1.099
Epoch 5 / 200: Loss = 1.099
Epoch 6 / 200: Loss = 1.099
Epoch 7 / 200: Loss = 1.098
Epoch 8 / 200: Loss = 1.098
Epoch 9 / 200: Loss = 1.098
Epoch 10 / 200: Loss = 1.098
Epoch 11 / 200: Loss = 1.098
Epoch 12 / 200: Loss = 1.098
Epoch 13 / 200: Loss = 1.097
Epoch 14 / 200: Loss = 1.096
Epoch 15 / 200: Loss = 1.092
Epoch 16 / 200: Loss = 1.079
Epoch 17 / 200: Loss = 1.045
Epoch 18 / 200: Loss = 1.004
Epoch 19 / 200: Loss = 0.984
Epoch 20 / 200: Loss = 0.971
Epoch 21 / 200: Loss = 0.960
Epoch 22 / 200: Loss = 0.951
Epoch 23 / 200: Loss = 0.946
Epoch 24 / 200: Loss = 0.938
Epoch 25 / 200: Loss = 0.931
Epoch 26 / 200: Loss = 0.924
Epoch 27 / 200: Loss = 0.917
Epoch 28 / 200: Loss = 0.912
Epoch 29 / 200: Loss = 0.907
Epoch 30 / 200: Loss = 0.900
Epoch 31 / 200: Loss = 0.897
Epoch 32 / 200: Loss = 0.890
Epoch 33 / 200: Loss = 0.885
Epoch 34 / 200: Loss = 0.877
Epoch 35 / 200: Loss = 

In [70]:
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Train dataset loss : 0.7149788298104939
Train dataset acc :82.38075971603394%
Test dataset loss : 0.7318928241729736
Test dataset acc :76.21093988418579%


([tensor([2, 2, 2, 0, 2, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1, 2, 1,
          0, 2, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0, 2, 1, 1, 0, 1, 0, 0, 1, 2, 0,
          1, 0, 2, 2, 0, 2, 0, 1, 0, 2, 1, 1, 0, 1, 2, 1, 0, 0, 2, 0, 1, 1, 2, 1,
          0, 1, 2, 0, 2, 0, 2, 2, 0, 1, 2, 2, 0, 0, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2,
          1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 0, 0,
          1, 1, 0, 1, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 2, 1, 0, 2,
          2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 2, 2, 2, 1, 0, 0, 1, 2, 0, 2, 1, 1, 1,
          1, 0, 2, 1, 1, 2, 1, 0, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 1, 2, 1,
          1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 1, 1, 0, 2, 1, 2, 0,
          1, 2, 2, 0, 1, 1, 1, 0, 2, 1, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 0,
          1, 1, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 0, 0, 1], device='cuda:0'),
  tensor([0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0,
          0, 1, 1, 0,

In [71]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 200, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 200: Loss = 1.099
Epoch 2 / 200: Loss = 1.098
Epoch 3 / 200: Loss = 1.053
Epoch 4 / 200: Loss = 0.929
Epoch 5 / 200: Loss = 0.862
Epoch 6 / 200: Loss = 0.818
Epoch 7 / 200: Loss = 0.789
Epoch 8 / 200: Loss = 0.779
Epoch 9 / 200: Loss = 0.775
Epoch 10 / 200: Loss = 0.759
Epoch 11 / 200: Loss = 0.754
Epoch 12 / 200: Loss = 0.748
Epoch 13 / 200: Loss = 0.762
Epoch 14 / 200: Loss = 0.760
Epoch 15 / 200: Loss = 0.746
Epoch 16 / 200: Loss = 0.740
Epoch 17 / 200: Loss = 0.737
Epoch 18 / 200: Loss = 0.737
Epoch 19 / 200: Loss = 0.744
Epoch 20 / 200: Loss = 0.740
Epoch 21 / 200: Loss = 0.749
Epoch 22 / 200: Loss = 0.733
Epoch 23 / 200: Loss = 0.736
Epoch 24 / 200: Loss = 0.728
Epoch 25 / 200: Loss = 0.727
Epoch 26 / 200: Loss = 0.742
Epoch 27 / 200: Loss = 0.731
Epoch 28 / 200: Loss = 0.728
Epoch 29 / 200: Loss = 0.727
Epoch 30 / 200: Loss = 0.725
Epoch 31 / 200: Loss = 0.729
Epoch 32 / 200: Loss = 0.725
Epoch 33 / 200: Loss = 0.726
Epoch 34 / 200: Loss = 0.725
Epoch 35 / 200: Loss = 

([tensor([1, 0, 0, 0, 2, 2, 1, 1, 2, 1, 1, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 1, 1, 0,
          1, 0, 1, 1, 2, 0, 0, 2, 0, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 0, 0, 1, 0, 0,
          1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 0, 1, 2, 0, 0, 2, 0, 0, 1, 1, 0, 2, 2, 1,
          0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2,
          0, 2, 0, 2, 0, 0, 0, 2, 1, 1, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0,
          0, 2, 2, 2, 1, 0, 0, 1, 2, 0, 2, 0, 1, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 2,
          1, 2, 0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0,
          0, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 1, 1, 1, 0, 2, 0, 0, 2, 1, 0, 1,
          2, 1, 0, 2, 1, 1, 1, 2, 0, 2, 2, 1, 0, 0, 2, 0, 0, 0, 1, 1, 2, 1, 0, 0,
          2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 2, 2, 1, 0, 0, 1, 1, 1, 1,
          0, 2, 0, 1, 2, 0, 1, 2, 0, 1, 1, 2, 0, 0, 0, 2], device='cuda:0'),
  tensor([2, 1, 1, 2, 2, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 0, 1, 2, 0, 1, 2, 0, 0,
          1, 0, 1, 2,

# no weight

In [303]:
eeg_input, eeg_output = generation_cnn_data(device, 'noweight')
dataset = MakeDataset(eeg_input, eeg_output)
train_dataset, test_dataset = random_split(dataset, 
                                           [round(eeg_output.size()[0]*0.8),
                                            round(eeg_output.size()[0]*0.2)])

trainloader = DataLoader(train_dataset, 
                         batch_size=256, 
                         shuffle=True)

testloader = DataLoader(test_dataset,
                        batch_size = 256,
                        shuffle=True)

In [74]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 200, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 200: Loss = 1.099
Epoch 2 / 200: Loss = 1.092
Epoch 3 / 200: Loss = 1.026
Epoch 4 / 200: Loss = 0.911
Epoch 5 / 200: Loss = 0.837
Epoch 6 / 200: Loss = 0.815
Epoch 7 / 200: Loss = 0.780
Epoch 8 / 200: Loss = 0.747
Epoch 9 / 200: Loss = 0.707
Epoch 10 / 200: Loss = 0.696
Epoch 11 / 200: Loss = 0.686
Epoch 12 / 200: Loss = 0.684
Epoch 13 / 200: Loss = 0.685
Epoch 14 / 200: Loss = 0.679
Epoch 15 / 200: Loss = 0.680
Epoch 16 / 200: Loss = 0.687
Epoch 17 / 200: Loss = 0.674
Epoch 18 / 200: Loss = 0.675
Epoch 19 / 200: Loss = 0.685
Epoch 20 / 200: Loss = 0.681
Epoch 21 / 200: Loss = 0.681
Epoch 22 / 200: Loss = 0.674
Epoch 23 / 200: Loss = 0.677
Epoch 24 / 200: Loss = 0.669
Epoch 25 / 200: Loss = 0.666
Epoch 26 / 200: Loss = 0.666
Epoch 27 / 200: Loss = 0.666
Epoch 28 / 200: Loss = 0.665
Epoch 29 / 200: Loss = 0.669
Epoch 30 / 200: Loss = 0.665
Epoch 31 / 200: Loss = 0.669
Epoch 32 / 200: Loss = 0.666
Epoch 33 / 200: Loss = 0.667
Epoch 34 / 200: Loss = 0.662
Epoch 35 / 200: Loss = 

([tensor([2, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0,
          2, 0, 0, 1, 0, 2, 1, 0, 1, 1, 2, 1, 0, 0, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1,
          0, 2, 0, 1, 0, 1, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 1, 2, 1, 1, 2, 1, 2, 2,
          0, 0, 2, 2, 1, 2, 0, 2, 0, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0,
          0, 2, 2, 0, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 2, 1, 0, 0, 0, 0, 1, 1,
          0, 1, 2, 1, 1, 1, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 0, 2, 0, 2, 0, 1,
          0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 2, 1, 1, 1, 2, 1, 2, 0, 0, 0, 0, 2, 2, 2,
          0, 0, 2, 0, 1, 2, 1, 0, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 0, 0, 2, 0, 2, 1,
          2, 0, 2, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 1, 1, 2, 0, 0, 1, 0,
          0, 1, 2, 2, 1, 2, 0, 1, 2, 0, 0, 1, 2, 1, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2,
          1, 0, 1, 1, 1, 1, 2, 0, 0, 2, 2, 1, 0, 0, 2, 2], device='cuda:0'),
  tensor([1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2,
          2, 0, 1, 1,

In [307]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 1000, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 1000: Loss = 1.099
Epoch 2 / 1000: Loss = 1.092
Epoch 3 / 1000: Loss = 1.024
Epoch 4 / 1000: Loss = 0.927
Epoch 5 / 1000: Loss = 0.844
Epoch 6 / 1000: Loss = 0.799
Epoch 7 / 1000: Loss = 0.756
Epoch 8 / 1000: Loss = 0.709
Epoch 9 / 1000: Loss = 0.692
Epoch 10 / 1000: Loss = 0.686
Epoch 11 / 1000: Loss = 0.682
Epoch 12 / 1000: Loss = 0.673
Epoch 13 / 1000: Loss = 0.675
Epoch 14 / 1000: Loss = 0.668
Epoch 15 / 1000: Loss = 0.672
Epoch 16 / 1000: Loss = 0.665
Epoch 17 / 1000: Loss = 0.660
Epoch 18 / 1000: Loss = 0.669
Epoch 19 / 1000: Loss = 0.666
Epoch 20 / 1000: Loss = 0.669
Epoch 21 / 1000: Loss = 0.661
Epoch 22 / 1000: Loss = 0.659
Epoch 23 / 1000: Loss = 0.659
Epoch 24 / 1000: Loss = 0.659
Epoch 25 / 1000: Loss = 0.659
Epoch 26 / 1000: Loss = 0.657
Epoch 27 / 1000: Loss = 0.659
Epoch 28 / 1000: Loss = 0.661
Epoch 29 / 1000: Loss = 0.659
Epoch 30 / 1000: Loss = 0.662
Epoch 31 / 1000: Loss = 0.658
Epoch 32 / 1000: Loss = 0.655
Epoch 33 / 1000: Loss = 0.659
Epoch 34 / 1000: Lo

([tensor([2, 0, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1, 1, 1, 0,
          1, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
          0, 0, 1, 0, 0, 1, 1, 0, 2, 1, 2, 0, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 0, 1,
          1, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 0, 1, 0,
          0, 2, 1, 1, 0, 2, 1, 0, 2, 0, 2, 1, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 1,
          0, 1, 0, 2, 0, 2, 1, 2, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 0, 1, 0,
          0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 2, 2, 2, 1, 0,
          0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0,
          1, 0, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 0, 2, 1, 2, 1, 1, 1,
          1, 1, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0, 0, 2, 1, 1, 0, 2, 0, 2, 1,
          2, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0], device='cuda:0'),
  tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 1, 2, 2,
          1, 1, 1, 2,

In [75]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 1000, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 1000: Loss = 1.099
Epoch 2 / 1000: Loss = 1.095
Epoch 3 / 1000: Loss = 1.024
Epoch 4 / 1000: Loss = 0.903
Epoch 5 / 1000: Loss = 0.767
Epoch 6 / 1000: Loss = 0.709
Epoch 7 / 1000: Loss = 0.698
Epoch 8 / 1000: Loss = 0.693
Epoch 9 / 1000: Loss = 0.690
Epoch 10 / 1000: Loss = 0.684
Epoch 11 / 1000: Loss = 0.686
Epoch 12 / 1000: Loss = 0.681
Epoch 13 / 1000: Loss = 0.679
Epoch 14 / 1000: Loss = 0.675
Epoch 15 / 1000: Loss = 0.680
Epoch 16 / 1000: Loss = 0.674
Epoch 17 / 1000: Loss = 0.673
Epoch 18 / 1000: Loss = 0.671
Epoch 19 / 1000: Loss = 0.675
Epoch 20 / 1000: Loss = 0.670
Epoch 21 / 1000: Loss = 0.670
Epoch 22 / 1000: Loss = 0.673
Epoch 23 / 1000: Loss = 0.671
Epoch 24 / 1000: Loss = 0.670
Epoch 25 / 1000: Loss = 0.665
Epoch 26 / 1000: Loss = 0.674
Epoch 27 / 1000: Loss = 0.674
Epoch 28 / 1000: Loss = 0.669
Epoch 29 / 1000: Loss = 0.667
Epoch 30 / 1000: Loss = 0.665
Epoch 31 / 1000: Loss = 0.661
Epoch 32 / 1000: Loss = 0.667
Epoch 33 / 1000: Loss = 0.669
Epoch 34 / 1000: Lo

([tensor([2, 2, 1, 2, 0, 2, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0,
          1, 2, 1, 2, 1, 2, 1, 0, 2, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 0, 2, 0, 2, 1,
          1, 1, 0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 1, 2, 0, 1, 1, 0, 2, 1, 2, 2, 1, 1,
          0, 1, 1, 1, 2, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 2, 1,
          0, 0, 2, 1, 1, 0, 0, 2, 1, 0, 0, 1, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0,
          2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 2, 2, 0, 1, 2, 1, 0,
          1, 0, 1, 2, 1, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 1, 2,
          2, 2, 1, 1, 1, 0, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 2, 2, 1, 0, 1, 0, 2, 0,
          2, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 0, 1, 2, 1,
          1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 0, 2, 2, 0,
          0, 0, 1, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0, 2, 2, 2], device='cuda:0'),
  tensor([0, 2, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 1, 1, 1, 0, 0, 0, 2, 1, 2,
          1, 2, 1, 0,

In [302]:
model = CNNEEG_32_2(32)
his = train_CNNEEG(model, trainloader, 200, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 200: Loss = 1.099
Epoch 2 / 200: Loss = 1.076
Epoch 3 / 200: Loss = 0.969
Epoch 4 / 200: Loss = 0.849
Epoch 5 / 200: Loss = 0.726
Epoch 6 / 200: Loss = 0.709
Epoch 7 / 200: Loss = 0.705
Epoch 8 / 200: Loss = 0.696
Epoch 9 / 200: Loss = 0.695
Epoch 10 / 200: Loss = 0.706
Epoch 11 / 200: Loss = 0.692
Epoch 12 / 200: Loss = 0.687
Epoch 13 / 200: Loss = 0.686
Epoch 14 / 200: Loss = 0.684
Epoch 15 / 200: Loss = 0.681
Epoch 16 / 200: Loss = 0.691
Epoch 17 / 200: Loss = 0.689
Epoch 18 / 200: Loss = 0.689
Epoch 19 / 200: Loss = 0.689
Epoch 20 / 200: Loss = 0.687
Epoch 21 / 200: Loss = 0.679
Epoch 22 / 200: Loss = 0.679
Epoch 23 / 200: Loss = 0.686
Epoch 24 / 200: Loss = 0.683
Epoch 25 / 200: Loss = 0.676
Epoch 26 / 200: Loss = 0.682
Epoch 27 / 200: Loss = 0.691
Epoch 28 / 200: Loss = 0.684
Epoch 29 / 200: Loss = 0.680
Epoch 30 / 200: Loss = 0.677
Epoch 31 / 200: Loss = 0.676
Epoch 32 / 200: Loss = 0.677
Epoch 33 / 200: Loss = 0.674
Epoch 34 / 200: Loss = 0.673
Epoch 35 / 200: Loss = 

([tensor([0, 0, 1, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0, 1, 2, 1, 0, 0,
          0, 2, 1, 2, 2, 1, 2, 0, 0, 1, 1, 1, 0, 2, 2, 2, 0, 2, 1, 0, 1, 1, 0, 1,
          1, 2, 1, 2, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1,
          2, 2, 0, 2, 2, 1, 2, 0, 2, 1, 0, 0, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1,
          2, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1,
          0, 0, 0, 2, 2, 0, 0, 1, 1, 2, 2, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 1, 1,
          2, 2, 1, 2, 2, 0, 1, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 2,
          2, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 2, 0, 1, 1, 0, 2, 0, 1, 2, 0, 0, 2, 2,
          1, 1, 0, 2, 1, 1, 2, 1, 0, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 2,
          0, 0, 0, 1, 1, 2, 2, 2, 0, 2, 2, 1, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 2,
          1, 2, 2, 1, 1, 1, 2, 2, 0, 2, 1, 1, 2, 1, 2, 2], device='cuda:0'),
  tensor([2, 1, 0, 1, 1, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 2, 2, 0,
          2, 2, 1, 2,

In [304]:
model = CNNEEG_32_2(32)
his = train_CNNEEG(model, trainloader, 200, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 200: Loss = 1.099
Epoch 2 / 200: Loss = 1.089
Epoch 3 / 200: Loss = 0.998
Epoch 4 / 200: Loss = 0.906
Epoch 5 / 200: Loss = 0.828
Epoch 6 / 200: Loss = 0.799
Epoch 7 / 200: Loss = 0.751
Epoch 8 / 200: Loss = 0.713
Epoch 9 / 200: Loss = 0.710
Epoch 10 / 200: Loss = 0.709
Epoch 11 / 200: Loss = 0.689
Epoch 12 / 200: Loss = 0.700
Epoch 13 / 200: Loss = 0.695
Epoch 14 / 200: Loss = 0.696
Epoch 15 / 200: Loss = 0.681
Epoch 16 / 200: Loss = 0.681
Epoch 17 / 200: Loss = 0.681
Epoch 18 / 200: Loss = 0.683
Epoch 19 / 200: Loss = 0.682
Epoch 20 / 200: Loss = 0.679
Epoch 21 / 200: Loss = 0.677
Epoch 22 / 200: Loss = 0.674
Epoch 23 / 200: Loss = 0.682
Epoch 24 / 200: Loss = 0.675
Epoch 25 / 200: Loss = 0.677
Epoch 26 / 200: Loss = 0.678
Epoch 27 / 200: Loss = 0.674
Epoch 28 / 200: Loss = 0.677
Epoch 29 / 200: Loss = 0.682
Epoch 30 / 200: Loss = 0.673
Epoch 31 / 200: Loss = 0.676
Epoch 32 / 200: Loss = 0.674
Epoch 33 / 200: Loss = 0.670
Epoch 34 / 200: Loss = 0.674
Epoch 35 / 200: Loss = 

([tensor([1, 2, 2, 1, 0, 0, 0, 1, 2, 1, 2, 2, 1, 0, 2, 0, 1, 2, 0, 0, 2, 0, 2, 0,
          1, 0, 0, 2, 1, 2, 0, 1, 1, 2, 1, 1, 2, 0, 0, 0, 2, 2, 1, 2, 2, 0, 1, 0,
          0, 2, 2, 0, 1, 0, 1, 1, 2, 0, 2, 0, 2, 2, 2, 1, 1, 2, 1, 0, 1, 1, 0, 2,
          2, 1, 0, 0, 1, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 2,
          1, 1, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 1, 1, 2, 0, 2, 2, 2, 1, 0, 2, 0, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 2,
          2, 2, 0, 0, 0, 0, 0, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 2, 2, 1, 2,
          0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 1, 2,
          1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0,
          0, 2, 1, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 0, 1, 0, 1, 0, 2, 0, 0,
          1, 1, 0, 2, 0, 2, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0], device='cuda:0'),
  tensor([0, 1, 1, 0, 2, 0, 0, 1, 1, 0, 0, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 0, 1, 2,
          1, 1, 1, 1,

In [305]:
model = CNNEEG_32_2(32)
his = train_CNNEEG(model, trainloader, 1000, 0.0001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 1000: Loss = 1.099
Epoch 2 / 1000: Loss = 1.099
Epoch 3 / 1000: Loss = 1.099
Epoch 4 / 1000: Loss = 1.099
Epoch 5 / 1000: Loss = 1.099
Epoch 6 / 1000: Loss = 1.099
Epoch 7 / 1000: Loss = 1.098
Epoch 8 / 1000: Loss = 1.098
Epoch 9 / 1000: Loss = 1.097
Epoch 10 / 1000: Loss = 1.095
Epoch 11 / 1000: Loss = 1.081
Epoch 12 / 1000: Loss = 1.030
Epoch 13 / 1000: Loss = 0.989
Epoch 14 / 1000: Loss = 0.973
Epoch 15 / 1000: Loss = 0.961
Epoch 16 / 1000: Loss = 0.951
Epoch 17 / 1000: Loss = 0.943
Epoch 18 / 1000: Loss = 0.934
Epoch 19 / 1000: Loss = 0.926
Epoch 20 / 1000: Loss = 0.917
Epoch 21 / 1000: Loss = 0.910
Epoch 22 / 1000: Loss = 0.901
Epoch 23 / 1000: Loss = 0.893
Epoch 24 / 1000: Loss = 0.883
Epoch 25 / 1000: Loss = 0.875
Epoch 26 / 1000: Loss = 0.870
Epoch 27 / 1000: Loss = 0.866
Epoch 28 / 1000: Loss = 0.863
Epoch 29 / 1000: Loss = 0.861
Epoch 30 / 1000: Loss = 0.857
Epoch 31 / 1000: Loss = 0.855
Epoch 32 / 1000: Loss = 0.852
Epoch 33 / 1000: Loss = 0.852
Epoch 34 / 1000: Lo

([tensor([2, 1, 1, 1, 2, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 0, 2, 1, 0, 1, 1,
          1, 2, 0, 2, 0, 1, 2, 1, 2, 1, 1, 2, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 2,
          1, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 1, 1, 1, 0, 2, 2, 1, 0, 0, 2, 2, 0,
          2, 0, 1, 1, 0, 0, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 2,
          1, 2, 1, 0, 2, 1, 0, 2, 0, 1, 0, 2, 1, 2, 0, 0, 1, 0, 2, 2, 0, 0, 1, 1,
          0, 2, 1, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 2,
          0, 0, 0, 2, 0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 1,
          2, 2, 1, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 1, 0, 2, 2, 0, 0, 0, 1,
          2, 0, 0, 1, 0, 0, 2, 0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 1, 2, 0, 0, 0, 2, 0,
          1, 2, 1, 0, 0, 2, 1, 2, 0, 2, 2, 2, 1, 1, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2,
          1, 0, 0, 1, 1, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], device='cuda:0'),
  tensor([2, 2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0, 1, 2, 1, 0, 0, 1,
          0, 0, 2, 2,

In [306]:
model = CNNEEG_32_2(32)
his = train_CNNEEG(model, trainloader, 1000, 0.001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 1000: Loss = 1.098
Epoch 2 / 1000: Loss = 1.062
Epoch 3 / 1000: Loss = 0.977
Epoch 4 / 1000: Loss = 0.906
Epoch 5 / 1000: Loss = 0.880
Epoch 6 / 1000: Loss = 0.834
Epoch 7 / 1000: Loss = 0.795
Epoch 8 / 1000: Loss = 0.755
Epoch 9 / 1000: Loss = 0.708
Epoch 10 / 1000: Loss = 0.699
Epoch 11 / 1000: Loss = 0.698
Epoch 12 / 1000: Loss = 0.703
Epoch 13 / 1000: Loss = 0.694
Epoch 14 / 1000: Loss = 0.692
Epoch 15 / 1000: Loss = 0.685
Epoch 16 / 1000: Loss = 0.679
Epoch 17 / 1000: Loss = 0.684
Epoch 18 / 1000: Loss = 0.691
Epoch 19 / 1000: Loss = 0.683
Epoch 20 / 1000: Loss = 0.681
Epoch 21 / 1000: Loss = 0.677
Epoch 22 / 1000: Loss = 0.680
Epoch 23 / 1000: Loss = 0.676
Epoch 24 / 1000: Loss = 0.680
Epoch 25 / 1000: Loss = 0.676
Epoch 26 / 1000: Loss = 0.675
Epoch 27 / 1000: Loss = 0.676
Epoch 28 / 1000: Loss = 0.680
Epoch 29 / 1000: Loss = 0.677
Epoch 30 / 1000: Loss = 0.681
Epoch 31 / 1000: Loss = 0.675
Epoch 32 / 1000: Loss = 0.673
Epoch 33 / 1000: Loss = 0.681
Epoch 34 / 1000: Lo

([tensor([2, 1, 1, 2, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0,
          1, 2, 1, 0, 0, 2, 1, 2, 1, 2, 0, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
          0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 2, 2, 0, 1, 2, 2, 1, 1, 2, 0, 2, 2, 0, 0,
          2, 1, 0, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 0, 0, 2, 2, 0, 1,
          0, 2, 1, 1, 2, 1, 1, 1, 0, 1, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 1, 2, 1, 2,
          1, 2, 0, 1, 0, 1, 0, 2, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 0, 0, 0, 1, 1, 1,
          1, 0, 0, 2, 1, 0, 1, 0, 0, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 2, 1, 1, 0, 2,
          0, 0, 0, 2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 1, 2, 0, 2, 1, 0, 1, 2, 2, 2, 2,
          0, 0, 0, 2, 1, 1, 1, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 2, 2, 2, 0, 1,
          2, 2, 0, 2, 2, 0, 0, 0, 1, 1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1,
          1, 2, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 0, 1, 0, 1], device='cuda:0'),
  tensor([0, 2, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 2,
          1, 0, 0, 1,

# nonscale (안봐도됨) 

In [122]:
def total_dataset_nonscale(weight):
    tasks = ['10', '20', '1back', '3back']

    task_datas = []
    task_labels = []

    for task in tasks:
        folder_path = './seq_data/nonscale/' + weight + '/' + task +'/'
        task_data = np.load(folder_path + 'total_split_datas.npy')
        task_label = np.load(folder_path + 'total_split_labels.npy')

        ind_list = [i for i in range(len(task_data))]
        random.shuffle(ind_list)

        if task == '20':
            task_data = task_data[ind_list[:4000], :, :]
            task_label = task_label[ind_list[:4000]]
        elif task == '3back':
            task_data = task_data[ind_list[:4000], :, :]
            task_label = task_label[ind_list[:4000]]
        else:
            task_data = task_data[ind_list[:2000], :, :]
            task_label = task_label[ind_list[:2000]]
        
        task_datas.append(task_data)
        task_labels.append(task_label)
    
    total_datas = np.concatenate(task_datas, axis=0)
    total_datas = (total_datas - np.min(np.min(total_datas, axis = 2),axis = 0).reshape(1,32,1)) / (np.max(np.max(total_datas, axis = 2),axis = 0).reshape(1,32,1) - np.min(np.min(total_datas, axis = 2),axis = 0).reshape(1,32,1))
    total_labels = np.concatenate(task_labels, axis=0)
    
    return total_datas, total_labels

In [124]:
def generation_nonscale_cnn_data(device, weight = 'weight'):
    total_datas, total_labels = total_dataset_nonscale(weight)
    eeg_input = torch.tensor(total_datas, dtype=float, device=device)
    eeg_output = torch.tensor(total_labels, dtype=int, device=device)
    
    return eeg_input, eeg_output

In [130]:
eeg_input, eeg_output = generation_nonscale_cnn_data(device, 'weight')
dataset = MakeDataset(eeg_input, eeg_output)
train_dataset, test_dataset = random_split(dataset, 
                                           [round(eeg_output.size()[0]*0.8),
                                            round(eeg_output.size()[0]*0.2)])

trainloader = DataLoader(train_dataset, 
                         batch_size=256, 
                         shuffle=True)

testloader = DataLoader(test_dataset,
                        batch_size = 256,
                        shuffle=True)

In [131]:
model = CNNEEG_32(32)
his = train_CNNEEG(model, trainloader, 100, 0.0001, device= device)
evaluation_eegcnn(model, trainloader, testloader, 256, device)

Epoch 1 / 100: Loss = 1.100
Epoch 2 / 100: Loss = 1.100
Epoch 3 / 100: Loss = 1.099
Epoch 4 / 100: Loss = 1.099
Epoch 5 / 100: Loss = 1.099
Epoch 6 / 100: Loss = 1.099
Epoch 7 / 100: Loss = 1.099
Epoch 8 / 100: Loss = 1.099
Epoch 9 / 100: Loss = 1.099
Epoch 10 / 100: Loss = 1.099
Epoch 11 / 100: Loss = 1.099
Epoch 12 / 100: Loss = 1.099
Epoch 13 / 100: Loss = 1.099
Epoch 14 / 100: Loss = 1.099
Epoch 15 / 100: Loss = 1.099
Epoch 16 / 100: Loss = 1.099
Epoch 17 / 100: Loss = 1.099
Epoch 18 / 100: Loss = 1.099
Epoch 19 / 100: Loss = 1.099
Epoch 20 / 100: Loss = 1.099
Epoch 21 / 100: Loss = 1.099
Epoch 22 / 100: Loss = 1.099
Epoch 23 / 100: Loss = 1.099
Epoch 24 / 100: Loss = 1.099
Epoch 25 / 100: Loss = 1.099
Epoch 26 / 100: Loss = 1.099
Epoch 27 / 100: Loss = 1.099
Epoch 28 / 100: Loss = 1.099
Epoch 29 / 100: Loss = 1.099
Epoch 30 / 100: Loss = 1.099
Epoch 31 / 100: Loss = 1.099
Epoch 32 / 100: Loss = 1.099
Epoch 33 / 100: Loss = 1.099
Epoch 34 / 100: Loss = 1.099
Epoch 35 / 100: Loss = 

([tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0'),
  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2,